## Main Hartree Code
<br>
Hartree-Fock Computational Chemistry Method implemented in Python as described in <u>Modern Quantum Chemistry Introduction to Advanced Electronic Structure Theory</u>, by Attila Szabo and Neil S. Ostlund. <br> <br>
Throughout the rest of the modules in this notebook, the entire text of Modern Quantum Chemistry will simply be refered to as "Szabo" for the sake of brevity. 
<br> <br>
Program is limited to molecules consisting of hydrogen and helium with only even numbers of electrons. Required user input is the atomic number, 3D space location, and electron number for each atom. A basis set specification is also required. <br>
<br>
The program makes use of Hartree Atomic Units such that the distances and lengths are described in Bohr Radius, energy is in Hartrees and mass is in Hartree Atomic Units, such that the mass of a proton is equal to 1836 atomic units. More information on Hartree Atomic Units can be found on page 41 of Szabo. 

In [1]:
#Python Implementation of the Hartree Fock Method
#Procedures listed in the code follow as described in Modern Quantum Chemistry: 
#Introduction to Advanced Electronic Structure Theory, By Attila Szabo and Neil S. Ostlund
import math
import sys
import numpy as np
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from notebookImporter import importNotebook

#import integrals notebook for the hartree method
integrals = importNotebook("Hartree_Integrals")
scf = importNotebook("Hartree_SCF")

#define SCF convergence critera, and max number of iteration cycles
SCF_CONVERGENCE = pow(10, -100)
MAX_ITERATIONS = 100

#Step 1
#Specify Molecules, Nuclear Coordinates, and Charge of the nucli Number of Electrons,

#generate an h2 atom with a distance of 1.4 AU to compare with Szabo pg. 160

#R is in units of Bohr Radius 
R = 1.4
system = molecule()
system.addAtom(atom(vector(1,1,1), 1, 1))
system.addAtom(atom(vector(1,1,1 + R), 1, 1))

#add a basis set
system.addBasis("STO-3G")

system.display()

Molecule
Basis Set: STO-3G
Total Number of Electrons: 2
Atoms: 
     Atomic Number: 1, Electrons: 1, Coordinate: X: 1 Y: 1 Z: 1
     Atomic Number: 1, Electrons: 1, Coordinate: X: 1 Y: 1 Z: 2.4


In [2]:
#Step 2
#Calculate Integrals
#Overlap, KE, Nuclear Attraaction, and Electron Repulsion
S = integrals.overlap(system)
print("Overlap Matrix: ")
print(np.matrix(S))
print()

T = integrals.kineticEnergy(system)
print("Electron Kinetic Energy Matrix: ")
print(np.matrix(T))
print()

V = integrals.nuclearAttraction(system)
for index, atom in enumerate(V):
    print("Nucli " + str(index) + "-Electron Attraction Matrix: ")
    print(np.matrix(atom))
print()

electronRepulsion = integrals.electronElectronRepulsion(system)
print("Electron Repulsion Tensor: ")
print(np.array(electronRepulsion))
print()

#Form the electronic hamiltonian
H = np.matrix(T)

#add in all of the nuclear attractions matricies to the hamiltonian
for atom in V:
    H += np.matrix(atom)
        
print("Electronic Hamiltonian :")
print(H)
print()

Overlap Matrix: 
[[1.         0.65931821]
 [0.65931821 1.        ]]

Electron Kinetic Energy Matrix: 
[[0.76003188 0.23645466]
 [0.23645466 0.76003188]]

Nucli 0-Electron Attraction Matrix: 
[[-1.22661373 -0.59741731]
 [-0.59741731 -0.65382716]]
Nucli 1-Electron Attraction Matrix: 
[[-0.65382716 -0.59741731]
 [-0.59741731 -1.22661373]]

Electron Repulsion Tensor: 
[[[[0.77460594 0.44410766]
   [0.44410766 0.56967593]]

  [[0.44410766 0.29702854]
   [0.29702854 0.44410766]]]


 [[[0.44410766 0.29702854]
   [0.29702854 0.44410766]]

  [[0.56967593 0.44410766]
   [0.44410766 0.77460594]]]]

Electronic Hamiltonian :
[[-1.12040901 -0.95837996]
 [-0.95837996 -1.12040901]]



In [3]:
#Prepare for the SCF procedure

#get size of the basis set 
size = len(S)

#compute the Transformation Matrix
X = scf.X(S, size)

#get guess Fock matrix, assume 2-electron term is equal to 0
F = H

In [4]:
# SCF Procedure 

#init list to store the energy from each iteration
#as well as a boolean to signify whether the loop has converged
E = []
converged = False

while( not converged ):
  
    #diagnolze the Fock matrix and convert it to MO basis 
    F = X.transpose() * F * X 
    
    #diagnolize the Fock Matrix to obtain the MOs and the their respective energies
    MOEnergy, MO = np.linalg.eigh(F)
    
    #Transform the MO basis MOs to an AO basis
    C = X * MO
    
    #compute the electron density, the two electron term, and then use G to compute the new Fock matrix
    P = scf.densityMatrix(C, system.N, size)
    G = scf.G(electronRepulsion, P, size)
    F = H + G
    
    #compute the new expectation energy
    #Expectation Energy is in units of Hartrees
    E.append(scf.expectationEnergy(H, F, P))
    
    #check if at least two SCF iterations have occured
    #if more than two have occured, then check if the difference betweeen this E, 
    #and the previous E is less then the covergence value, if yes, end the SCF loop
    #if energy has not converged, check whether the max number of iterations have occured so far
    sizeE = len(E)
    if(len(E) > 2):
        if(abs(E[sizeE-2] - E[sizeE-1]) < SCF_CONVERGENCE):
            converged = True
        elif(sizeE > MAX_ITERATIONS):
            print("SCF Failed to Converge")
            break
    
    #compute total energy of the system including nuclear-nuclear repulsion
    totalE = E[sizeE-1] + scf.nuclearRepulsion(system)
    
    #display information about current SCF iteration to the user
    print("SCF Iteration #" + str(sizeE) + ", Electronic Energy: " + str(E[sizeE-1]) + " Hartrees, Total Energy: " + str(totalE) + " Hartrees")
    
print("-"*50)
print()
print("Final SCF Energy: " + str(E[sizeE-1]))
print()
print(MOEnergy)
print(H)

SCF Iteration #1, Electronic Energy: -1.8310000394614832 Hartrees, Total Energy: -1.116714325175769 Hartrees
SCF Iteration #2, Electronic Energy: -1.8310000394614832 Hartrees, Total Energy: -1.116714325175769 Hartrees
SCF Iteration #3, Electronic Energy: -1.8310000394614832 Hartrees, Total Energy: -1.116714325175769 Hartrees
--------------------------------------------------

Final SCF Energy: -1.8310000394614832

[-0.57820298  0.67026776]
[[-1.12040901 -0.95837996]
 [-0.95837996 -1.12040901]]
